In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emot
!pip install emoji
!pip install demoji
!pip install punct
!pip install contractions

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd

In [ ]:
nltk.download('omw-1.4')
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import re
import time
import string
import emoji
import demoji
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import TweetTokenizer
import regex
import nltk
from nltk.stem import SnowballStemmer
demoji.download_codes()
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.svm import LinearSVC

<ipython-input-6-06cbf3f148ab>:23: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


**Reading dataset**

In [ ]:
df_train=pd.read_csv('/content/drive/MyDrive/All_Task_Updated_DataSet/Fake/Task1/Fake_train.csv')
df_dev=pd.read_csv('/content/drive/MyDrive/All_Task_Updated_DataSet/Fake/Task1/Fake_dev.csv')
df_test=pd.read_csv('/content/drive/MyDrive/All_Task_Updated_DataSet/Fake/Task1/Fake_test_without_labels.csv')

In [ ]:
df_train.head()

,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ കലാപം പോലെ ആകുമോ.. സ്വന്തം ആളെ തന്നെ പിടിച്ചു അകത്തിടേണ്ടി വരുമോ?",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth manthrii,original


In [ ]:
df_train['label'].value_counts()

original    1658
Fake        1599
Name: label, dtype: int64

In [ ]:
df_dev['label'].value_counts()

original    409
Fake        406
Name: label, dtype: int64

**Preprocessing**

In [ ]:
#emot_object = emot.core.emot()
ps =PorterStemmer()
lemmatiser = WordNetLemmatizer()
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)
def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  text =re.sub("@ ?[A-Za-z0-9_]+", "", text)
  text= re.sub(r'\d+', '', text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

In [ ]:
#EXTRA PREPROCESSING CODE
import pandas as pd
import re
# from indicnlp.tokenize import indic_tokenize

def preprocess_code_mixed_text(text):
    english_pattern = r'[a-zA-Z]+'
    malayalam_pattern = r'[ം-ഃ അ-ഹ ാ-് ൦-൯]+'

    english_segments = re.findall(english_pattern, text)
    malayalam_segments = re.findall(malayalam_pattern, text)

    processed_english_segments = [segment.lower() for segment in english_segments]

    processed_malayalam_segments = []
    for segment in malayalam_segments:
        tokens = indic_tokenize.trivial_tokenize(segment)
        processed_malayalam_segments.append(' '.join(tokens))

    processed_text = ' '.join(processed_english_segments + processed_malayalam_segments)

    return processed_text


In [ ]:
import emoji
#import demoji
#demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

In [ ]:
df_train['clean_text']=df_train["text"].apply(lambda x:emo(x))
df_train["clean_text"]=df_train['clean_text'].apply(lambda X: preprocess(X))

In [ ]:
df_train.head()

,text,label,clean_text
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake,നല്ല അവതരണം സത്യം പുറത്തു വരട്ടെ
1,Masha Allah,Fake,masha allah
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ കലാപം പോലെ ആകുമോ.. സ്വന്തം ആളെ തന്നെ പിടിച്ചു അകത്തിടേണ്ടി വരുമോ?",Fake,അന്വേഷണം കഴിയുമ്പോൾ c യ്ക്ക് എതിരായ കലാപം പോലെ ആകുമോ സ്വന്തം ആളെ തന്നെ പിടിച്ചു അകത്തിടേണ്ടി വരുമോ
3,Illathentha avaru purath vidayittalland verenth,Fake,illathentha avaru purath vidayittalland verenth
4,Barana pakshathin matoru niyamam.nalla moyanth manthrii,original,barana pakshathin matoru niyamamnalla moyanth manthrii


In [ ]:
df_train.to_csv('/content/drive/MyDrive/cleaned_data1.csv')

In [ ]:
cdata=pd.read_csv('/content/drive/MyDrive/cleaned_data1.csv')

In [ ]:
cdata=cdata.drop('text', axis=1)

In [ ]:
cdata.to_csv('/content/drive/MyDrive/cleaned_data2.csv')

In [ ]:
cdata

,Unnamed: 0,label,clean_text
0,0,Fake,നല്ല അവതരണം സത്യം പുറത്തു വരട്ടെ
1,1,Fake,masha allah
2,2,Fake,അന്വേഷണം കഴിയുമ്പോൾ c യ്ക്ക് എതിരായ കലാപം പോലെ ആകുമോ സ്വന്തം ആളെ തന്നെ പിടിച്ചു അകത്തിടേണ്ടി വരുമോ
3,3,Fake,illathentha avaru purath vidayittalland verenth
4,4,original,barana pakshathin matoru niyamamnalla moyanth manthrii
...,...,...,...
3252,3252,original,വീണാ ജോർജ് എന്ന ഒരു ആരോഗ്യ മന്ത്രി വന്നതിനു ശേഷം കേരളത്തിന്റെ ആരോഗ്യ സ്ഥിതി വളരെ മോശവും പ്രഹസനവുമാണ്
3253,3253,original,ഇതാണ് പിണുവാതിര face tears joy face tears joy face tears joy
3254,3254,original,കേസ് എടുക്കണം hundred points thumbs
3255,3255,original,എല്ലാത്തിനേം സോപ്പിട്ട് കളിപ്പിച്ച് വിടണം


In [ ]:
df_dev['clean_text']=df_dev["text"].apply(lambda x:emo(x))
df_dev["clean_text"]=df_dev['clean_text'].apply(lambda X: preprocess(X))

In [ ]:
df_test['clean_text']=df_test["text"].apply(lambda x:emo(x))
df_test["clean_text"]=df_test['clean_text'].apply(lambda X: preprocess(X))

**Feature Extraction:**TF-IDF of Syllable ngrams

In [ ]:
!git clone https://github.com/libindic/syllabalizer.git
%cd syllabalizer
%run setup.py sdist

In [ ]:
!pip install dist/libindic-syllabifier*.tar.gz
!pip install libindic-utils

Processing ./dist/libindic-syllabifier-0.0.1.dev28.tar.gz
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-syllabifier: filename=libindic_syllabifier-0.0.1.dev28-py2.py3-none-any.whl size=8123 sha256=812bee3d0903c22ca89f48ff59216fc96f382247eb23b09a2691f1613e2617a8
  Stored in directory: /root/.cache/pip/wheels/d2/50/19/e2620530fc718493df9f5bca90d23db025cc864b9c7c0a98ef
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8466 sha256=4a4b70a8e34efa22b03de2dbb2d99e1a9e9d49657ddf4efc7152bc69f120b3a5
  Stored in directory: /root/.cache/pip/wheels/c0/72/43/0c779b79d708c78240beb3b0bb8f5ff3c2ab81c4e5271ea1aa
Successfully built libindic-syllabifier silpa_common
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-utils: filename=libindic_utils-1.0.3-py3-none-any.whl size=9686 sha256=e0807d6022dc4edfc786dc2fc70e2a65a09342185a12eca1a08e6ccc18e18c2a
  Stored in directory: /root/.cache/pip/w

In [ ]:
from libindic.syllabifier import Syllabifier
instance = Syllabifier()

In [ ]:
def syllable(word):
    try:
        sylbls = instance.syllabify(word)
    except:
        sylbls = list(word)
    return sylbls

In [ ]:
# generate word n-grams

def generate_N_grams(text, ngram=1):
    words = [word for word in text.split(" ")]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[words[i:] for i in range(0, ngram)])
    ngrams = [' '.join(ngram) for ngram in temp]
    return ngrams

In [ ]:
# import regex

# def custom_analyzer(text):
#     words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
#     for w in words:
#         yield w

In [ ]:
# generate syllable n-grams by specifying the value of n, by default ngram=1

def generate_cN_grams(word, ngram=1):
    sylbs = [sylbs for sylbs in syllable(word)]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[sylbs[i:] for i in range(0, ngram)])
    ngrams = [''.join(ngram) for ngram in temp]
    return ngrams

In [ ]:
# generate syllable n-grams in the range 'from' to 'to'. by default 'from'= 1

def ngrams(text, r_to, r_from=1):
    r_from = 1
    r_to = 3
    allngrams=[]
    for word in text.split():
        temp = []
        for i in range(r_from + r_to - 1):
            temp.extend(generate_cN_grams(word, i+1))
            allngrams.append(temp)
    n_grams = [item for sublist in allngrams for item in sublist]
  # merged = list(itertools.chain(allngrams))
    return n_grams

In [ ]:
def custom_analyzer(text):
    syngrams = ngrams(text,3)
    return syngrams

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer=custom_analyzer, max_features=5000)
tfidf_vectorizer.fit_transform(df_train['clean_text'])
train_tfidf =  tfidf_vectorizer.transform(df_train['clean_text'])
dev_tfidf =  tfidf_vectorizer.transform(df_dev['clean_text'])
test_tfidf =  tfidf_vectorizer.transform(df_test['clean_text'])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(train_tfidf, df_train['label'])
NB_val_pred=naive_bayes_classifier.predict(dev_tfidf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_dev['label'], NB_val_pred))

              precision    recall  f1-score   support

        Fake       0.81      0.86      0.83       406
    original       0.85      0.80      0.83       409

    accuracy                           0.83       815
   macro avg       0.83      0.83      0.83       815
weighted avg       0.83      0.83      0.83       815



In [ ]:
NB_pred = naive_bayes_classifier.predict(test_tfidf)

In [ ]:
from sklearn.svm import LinearSVC
LSVC =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
LSVC.fit(train_tfidf, df_train['label'])
val_pred=LSVC.predict(dev_tfidf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_dev['label'], val_pred))

              precision    recall  f1-score   support

        Fake       0.86      0.81      0.84       406
    original       0.82      0.87      0.85       409

    accuracy                           0.84       815
   macro avg       0.84      0.84      0.84       815
weighted avg       0.84      0.84      0.84       815



In [ ]:
LSVCTest_pred = LSVC.predict(test_tfidf)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log = LogisticRegression()
log.fit(train_tfidf, df_train['label'])
LR_pred = log.predict(dev_tfidf)
print(classification_report(df_dev['label'], LR_pred))

              precision    recall  f1-score   support

        Fake       0.82      0.83      0.82       406
    original       0.83      0.81      0.82       409

    accuracy                           0.82       815
   macro avg       0.82      0.82      0.82       815
weighted avg       0.82      0.82      0.82       815



In [ ]:
LR_pred_test = log.predict(test_tfidf)
LR_pred_test

array(['original', 'Fake', 'original', ..., 'original', 'Fake',
       'original'], dtype=object)

In [ ]:
import sklearn
from sklearn import svm

In [ ]:
sv = svm.SVC(kernel='linear')
sv.fit(train_tfidf, df_train['label'])
SVM_pred= sv.predict(dev_tfidf)
print(classification_report(df_dev['label'], SVM_pred))

              precision    recall  f1-score   support

        Fake       0.85      0.82      0.83       406
    original       0.82      0.85      0.84       409

    accuracy                           0.83       815
   macro avg       0.83      0.83      0.83       815
weighted avg       0.83      0.83      0.83       815



In [ ]:
SVM_pred_test= sv.predict(test_tfidf)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(train_tfidf, df_train['label'])
Knn_pred=knn.predict(dev_tfidf)
print(classification_report(df_dev['label'], Knn_pred))

              precision    recall  f1-score   support

        Fake       0.67      0.89      0.76       406
    original       0.84      0.56      0.67       409

    accuracy                           0.72       815
   macro avg       0.75      0.72      0.72       815
weighted avg       0.75      0.72      0.72       815



In [ ]:
Knn_pred_test=knn.predict(test_tfidf)

In [ ]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier(
    estimators=[('LinearSVCn', LSVC),('SVM', sv),('MultinomialNB', naive_bayes_classifier)],
    voting='hard')

In [ ]:
voting.fit(train_tfidf, df_train['label'])

VotingClassifier(estimators=[('LinearSVCn',
                              LinearSVC(class_weight='balanced', max_iter=10000,
                                        random_state=123)),
                             ('SVM', SVC(kernel='linear')),
                             ('MultinomialNB', MultinomialNB())])

In [ ]:
voting_pred= voting.predict(dev_tfidf)
print(classification_report(df_dev['label'], voting_pred))

              precision    recall  f1-score   support

        Fake       0.84      0.82      0.83       406
    original       0.82      0.85      0.84       409

    accuracy                           0.83       815
   macro avg       0.83      0.83      0.83       815
weighted avg       0.83      0.83      0.83       815



In [ ]:
test_voting_pred= voting.predict(test_tfidf)

In [ ]:
len(test_voting_pred)

1019